https://www.kaggle.com/competitions/amex-default-prediction/data

In [2]:
import gc
import os
import psutil
import pyarrow as pa
import time

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random

In [4]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\KonuTech\zoomcamp-capstone-01\notebooks


In [6]:
!ls -lah "C:\Users\KonuTech\zoomcamp-capstone-01\data"

total 36G
drwxr-xr-x 1 KonuTech 197121    0 Oct 29 22:46 .
drwxr-xr-x 1 KonuTech 197121    0 Oct 29 19:44 ..
drwxr-xr-x 1 KonuTech 197121    0 Oct 29 22:46 parquet_partitions
-rw-r--r-- 1 KonuTech 197121  60M May 20  2022 sample_submission.csv
-rw-r--r-- 1 KonuTech 197121  32G May 20  2022 test_data.csv
-rw-r--r-- 1 KonuTech 197121  16G May 20  2022 train_data.csv
-rw-r--r-- 1 KonuTech 197121 582M Oct 29 22:46 train_data.parquet
-rw-r--r-- 1 KonuTech 197121  30M May 20  2022 train_labels.csv
-rw-r--r-- 1 KonuTech 197121  27M Oct 29 22:42 train_labels.parquet


In [7]:
# Define the directory path
data_dir = os.path.join('C:\\', 'Users', 'KonuTech', 'zoomcamp-capstone-01', 'data')
data_dir

'C:\\Users\\KonuTech\\zoomcamp-capstone-01\\data'

In [8]:
# Step 4: Convert and save the combined DataFrame as a single Parquet file
combined_parquet_file = 'train_data.parquet'

In [9]:
# Step 6: EDA on the combined Parquet file
parquet_df = pd.read_parquet(os.path.join(data_dir, combined_parquet_file))  # Read the combined Parquet file

In [10]:
# Trigger garbage collection to clear unreferenced objects
gc.collect()

18

In [11]:
parquet_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 458913 entries, 0 to 5531438
Data columns (total 190 columns):
 #    Column       Dtype  
---   ------       -----  
 0    customer_ID  object 
 1    S_2          object 
 2    P_2          float64
 3    D_39         float64
 4    B_1          float64
 5    B_2          float64
 6    R_1          float64
 7    S_3          float64
 8    D_41         float64
 9    B_3          float64
 10   D_42         float64
 11   D_43         float64
 12   D_44         float64
 13   B_4          float64
 14   D_45         float64
 15   B_5          float64
 16   R_2          float64
 17   D_46         float64
 18   D_47         float64
 19   D_48         float64
 20   D_49         float64
 21   B_6          float64
 22   B_7          float64
 23   B_8          float64
 24   D_50         float64
 25   D_51         float64
 26   B_9          float64
 27   R_3          float64
 28   D_52         float64
 29   P_3          float64
 30   B_10         float64
 

In [12]:
def save_histograms(dataframe, save_path):

    start_time = time.time()
    
    numeric_columns = dataframe.select_dtypes(include=['number'])
    
    # Create the directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)
    
    for column in numeric_columns.columns:
        plt.figure(figsize=(8, 6))
        
        sns.histplot(dataframe[column], kde=True, color='blue', bins=20)
        plt.title(f'Histogram with KDE of {column}')
        plt.xlabel(column)
        plt.ylabel('Density')
        
        # Calculate the number of events (N) and format it with thousands separators
        num_events = len(dataframe[column])
        formatted_num_events = '{:,}'.format(num_events)
        
        # Add annotation for N in the top-middle part of the histogram
        plt.text(0.5, 0.95, f'N={formatted_num_events}', fontsize=12, ha='center', transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.7))
        
        # Save the histogram as a .jpg file with the column name
        file_name = f'{column}.jpg'
        file_path = os.path.join(save_path, file_name)
        plt.savefig(file_path)
        
        print(f'Saved: {file_name}')
        
        plt.close()  # Close the plot to avoid displaying in the notebook

        # Print time elapsed for each whole minute
        elapsed_time = time.time() - start_time
        if int(elapsed_time) % 60 == 0 or num_events == len(dataframe[column]) - 1:
            print(f'Time elapsed: {int(elapsed_time / 60)} minutes')

In [13]:
for column in parquet_df.columns:
    print("Column name:", column)
    print("Data type:", parquet_df[column].dtype)
    print("First five unique values:", parquet_df[column].unique()[:5])
    print("Count of unique values:", parquet_df[column].nunique())
    print("Count of NULL values:", parquet_df[column].isnull().sum(), "\n")

Column name: customer_ID
Data type: object
First five unique values: ['0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'
 '00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5'
 '00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1'
 '000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc'
 '00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed']
Count of unique values: 458913
Count of NULL values: 0 

Column name: S_2
Data type: object
First five unique values: ['2017-03-09' '2017-03-01' '2017-03-11' '2017-03-31' '2017-03-30']
Count of unique values: 396
Count of NULL values: 0 

Column name: P_2
Data type: float64
First five unique values: [0.93846872 0.92912192 0.87661549 0.56744185 0.93684158]
Count of unique values: 441415
Count of NULL values: 17498 

Column name: D_39
Data type: float64
First five unique values: [0.00173334 0.38278958 0.00146856 0.26847613 0.00657352]
Count of unique values: 458913
Count of NULL values

In [14]:
parquet_df.head().to_dict(orient='records')

[{'customer_ID': '0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a',
  'S_2': '2017-03-09',
  'P_2': 0.9384687191272548,
  'D_39': 0.0017333390041739,
  'B_1': 0.0087244509498605,
  'B_2': 1.0068382339663076,
  'R_1': 0.0092277222786689,
  'S_3': 0.1240351558143743,
  'D_41': 0.0087711319938824,
  'B_3': 0.0047092406313857,
  'D_42': nan,
  'D_43': nan,
  'D_44': 0.0006301348049115,
  'B_4': 0.0809863324662527,
  'D_45': 0.708906305121159,
  'B_5': 0.1706002293387026,
  'R_2': 0.0062040314303209,
  'D_46': 0.3585865793715965,
  'D_47': 0.525351040810055,
  'D_48': 0.255736073902975,
  'D_49': nan,
  'B_6': 0.0639022133803909,
  'B_7': 0.0594157330614109,
  'B_8': 0.006465576798311,
  'D_50': 0.1486979558199995,
  'D_51': 1.3358557940752642,
  'B_9': 0.0082067391252636,
  'R_3': 0.0014225021561254,
  'D_52': 0.2073338786110817,
  'P_3': 0.7364627260945562,
  'B_10': 0.0962188066642168,
  'D_53': nan,
  'S_5': 0.0233811220509184,
  'B_11': 0.0027680616648439,
  'S_6': 0.0

In [15]:
parquet_df.describe(include='all')

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
count,458913,458913,441415.000000,4.589130e+05,458913.000000,456897.000000,4.589130e+05,372973.000000,4.568970e+05,4.568970e+05,109289.000000,283185.000000,4.205530e+05,4.589130e+05,4.568960e+05,4.589130e+05,4.589130e+05,308101.000000,458913.000000,383086.000000,37322.000000,458680.000000,458913.000000,4.541490e+05,188980.000000,4.589130e+05,4.589130e+05,4.589130e+05,440525.000000,377077.000000,458913.000000,9.943100e+04,4.589130e+05,4.589130e+05,4.589130e+05,456897.000000,4.589130e+05,372973.000000,4.589130e+05,4.589130e+05,420633.000000,175846.000000,4.191180e+05,4.589130e+05,4.589130e+05,1.864220e+05,458913.000000,392376.000000,4.589130e+05,409819.000000,458293.000000,458913.000000,385787.000000,458913,380874,4.589130e+05,4.568970e+05,1.630150e+05,4.589130e+05,4.568970e+05,44044.000000,4.568970e+05,381312.000000,458913.000000,4.589130e+05,4.589130e+05,4.589130e+05,3.813460e+05,4.568970e+05,4.050800e+05,4.589130e+05,4.446510e+05,458913.000000,4.589130e+05,7174.000000,4.589130e+05,4.526560e+05,4.589130e+05,5.013900e+04,4.589130e+05,4.589130e+05,2.692600e+05,458293.000000,4.568970e+05,4.205530e+05,4.099530e+05,4.589130e+05,27027.000000,4.589130e+05,4.526560e+05,4.589130e+05,4.589130e+05,4.568970e+05,4.418080e+05,1.186910e+05,4.589130e+05,458913.000000,458913.000000,4.589130e+05,3.813460e+05,4.589130e+05,4.589130e+05,4.405250e+05,4.589130e+05,3.312300e+04,456897.000000,4.589130e+05,4.589130e+05,187.0,4.589130e+05,4.589130e+05,297.000000,458913.000000,4.589130e+05,4.589130e+05,4.589130e+05,4.589130e+05,4.588390e+05,4.589130e+05,4.568970e+05,4.405250e+05,4.589130e+05,4.589130e+05,3.915310e+05,4.589130e+05,4.589130e+05,4.589130e+05,4.589130e+05,4.589130e+05,4.589130e+05,456782.000000,458876.000000,456828.000000,458005.000000,4.589120e+05,4.284310e+05,4.097470e+05,4.097470e+05,187747.000000,3.721900e+04,4.097470e+05,4.589130e+05,458913.000000,4.843600e+04,431543.000000,456897.000000,2.140000e+03,4.579490e+05,1040.000000,1040.000000,1048.000000,456896.000000,4.589130e+05,3.106050e+05,3.925870e+05,392587.000000,3.925870e+05,392587.000000,392587.000000,3.925870e+05,3.925870e+05,392587.000000,392587.000000,392587.000000,3.925870e+05,392587.000000,3.925870e+05,414975.000000,4.589130e+05,4.097470e+05,4.097470e+05,4.589120e+05,5502.000000,4.097470e+05,4.097470e+05,37298.000000,4.282350e+05,4.589130e+05,12731.000000,1.273100e+04,12731.000000,1.273100e+04,12731.000000,4.097470e+05,4.284340e+05,4.097470e+05,70920.000000,4.097470e+05,4.284210e+05,4.097470e+05
unique,458913,396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [16]:
# Define the directory path
save_path = os.path.join('C:\\', 'Users', 'KonuTech', 'zoomcamp-capstone-01', 'eda', "histograms")
save_path

'C:\\Users\\KonuTech\\zoomcamp-capstone-01\\eda\\histograms'

In [17]:
save_histograms(parquet_df, save_path)

Saved: P_2.jpg
Saved: D_39.jpg
Saved: B_1.jpg
Saved: B_2.jpg
Saved: R_1.jpg
Saved: S_3.jpg
Saved: D_41.jpg
Saved: B_3.jpg
Saved: D_42.jpg
Saved: D_43.jpg
Saved: D_44.jpg
Saved: B_4.jpg
Saved: D_45.jpg
Saved: B_5.jpg
Saved: R_2.jpg
Saved: D_46.jpg
Saved: D_47.jpg
Saved: D_48.jpg
Saved: D_49.jpg
Saved: B_6.jpg
Saved: B_7.jpg
Saved: B_8.jpg
Saved: D_50.jpg
Saved: D_51.jpg
Saved: B_9.jpg
Saved: R_3.jpg
Saved: D_52.jpg
Saved: P_3.jpg
Saved: B_10.jpg
Saved: D_53.jpg
Saved: S_5.jpg
Saved: B_11.jpg
Saved: S_6.jpg
Saved: D_54.jpg
Saved: R_4.jpg
Saved: S_7.jpg
Saved: B_12.jpg
Saved: S_8.jpg
Saved: D_55.jpg
Saved: D_56.jpg
Saved: B_13.jpg
Saved: R_5.jpg
Saved: D_58.jpg
Saved: S_9.jpg
Saved: B_14.jpg
Saved: D_59.jpg
Saved: D_60.jpg
Saved: D_61.jpg
Saved: B_15.jpg
Time elapsed: 1 minutes
Saved: S_11.jpg
Saved: D_62.jpg
Saved: D_65.jpg
Saved: B_16.jpg
Saved: B_17.jpg
Saved: B_18.jpg
Saved: B_19.jpg
Saved: D_66.jpg
Saved: B_20.jpg
Saved: D_68.jpg
Saved: S_12.jpg
Saved: R_6.jpg
Saved: S_13.jpg
Saved: 